In [ ]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVector

In [ ]:
raw_pdf_elements = partition_pdf(
    filename= '/content/drive/MyDrive/Multimodal_RAG/AC-Aids.pdf',
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_image_block_output_dir=output_path
)

In [ ]:
text_elements = []
table_elements = []

text_summaries = []
table_summaries = []

summary_prompt = """
Summarize the following {elemnt_type}:
{element}
"""

In [ ]:
summary_chain = LLMChain(
    llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key = openai_api_key,
                     prompt = PromptTemplate.from_template(summary_prompt))
)

In [ ]:
for e in raw_pdf_elements:
  if 'CompositeElement' in repr(e):
    text_elements.append(e.text)
    summary = summary_chain.run(element_type = 'text', element = e.text)
    text_summaries.append(summary)

  elif 'Table' in repr(e):
    table_elements.append(e.text)
    summary = summary_chain.run(element_type = 'table', element = e.text)
    table_summaries.append(summary)